# DataFrame II

Los objetivos de aprendizaje son:

1. Columnas
    + ¿Qué es una columna?
    + Seleccionar
    + Modificar
    + Crear
    + .apply()
    + Eliminar
2. Gestión Datos Nullos
    + Drop 
    + Fill
3. Filtros
    + Condiciones Lógicas
    + .isin()
    + .isnull()
    + .between()
    + .query()


## 1. Columnas

### ¿Qué es una columna?

Desde una perspectiva de *data modeling*, una estructura tabular, tal y como lo es un DataFrame, nos ayuda a generar una representación abstracta de un objeto.


In [ ]:
import numpy as np
import pandas as pd

nba_df = pd.read_csv("./Data/pandas/nba.csv")
nba_df.head()

Este modelo de datos serviría para representar a jugadores de la NBA, en este sentido:

1. Los renglones/filas representan jugadores.
2. **Las columnas representan caracteristicas de los jugadores**.

Por ejemplo:

El índice 2, representaría a un jugador cuya característica `Name = "John Holland"`.

### Seleccionar

Para seleccionar una columna de un `DataFrame` usaremos el comando

```python
df[<nombre columna>]
````

In [ ]:
nba_df['Name']

In [ ]:
type(nba_df['Name'])

Una columna es en realidad un atributo de la clase DataFrame. Al ser una columna un atributo de la clase DataFrame, podemos acceder mediante la notación *dot*:

```Python
dataframe.<nombre_columna>
```

In [ ]:
nba_df.Team

La primera opción es más flexibe para situaciones en las que los nombres de las columnas no cumplen con las reglas para nombrar variables en Python.

Podemos seleccionar un grupo de columnas de la siguiente forma:

In [ ]:
nba_df[["Name", "Team"]].head()

In [ ]:
columnas = [col for col in nba_df.columns if "a" in col.lower()]
print("Seleccionaremos las columns: {} \nResultado:".format(columnas))
nba_df[columnas].head()

### Modificar

En ocasiones, queremos alterar los valores de una columna aplicando alguna operación o función a cada elemento a.k.a. *element-wise operations* o *broadcasting*.

In [ ]:
# Caso numérico 
nba_df[['Age']]


In [ ]:
nba_df[['Age']] + 1

En el caso de las `strings` podemos aplicar métodos a cada registro del `DataFram`, pero tendremos que hacerlo con un cambio.

Supongamos que queremos convertir a minúsculas la columna `Name`.

In [ ]:
nba_df['Name'].head()

In [ ]:
nba_df['Name'].lower()

In [ ]:
nba_df['Name'].str.lower()

### Crear

Podemos añadir columnas a un DataFrame de una manera muy sencilla, veamos cómo hacerlo


In [ ]:
nba_df.head()

In [ ]:
nba_df["Columna_Extra"] = "Esto es una Columna Extra"
nba_df.head()

Supongamos que queremos crear dos columnas a partir de la columna `Name`: `nombre` y `apellido`.

In [ ]:
# Preimero separamos el campo Name
nba_df['Name'].str.split().head()

In [ ]:
nba_df['nombre'] = nba_df['Name'].str.split().str[0]
nba_df['apellido'] = nba_df['Name'].str.split().str[1]
nba_df.head()

In [ ]:
# También es posible realizar operaciones entre columnas.
nba_df['Age'] / nba_df['Weight']

### .apply()

El método aplply nos permite aplicar funciones más complejas entre los valores de las columnas.

Supongamos que queremos re-escalar el valor de la columna `Salary` en función del valor de la columna `Position`.
Vamos los valores de la columna `Position`:


In [ ]:
nba_df['Position'].value_counts()

Ahora crearemos una función que:

1. Tendrá como input renglones de un DataFrae.
2. Extraerá los valores del DataFrame. 
3. Realizará $n$ operaciones con ellos y regresará un valor. 

In [ ]:
def position_salary(row: pd.Series):
    position = row[0]
    salary = row[1]
    
    if position == "PG":
        return round(salary/2,2)
    elif position =="PF":
        return round(salary * 0.3,2)
    else:
        return salary

In [ ]:
nba_df['Modified_Salary']= nba_df[['Position','Salary']].apply(position_salary, axis = 1)
nba_df[['Position','Salary','Modified_Salary']]

Pero... ¿Qué está pasando exactamente?

In [ ]:
def test_by_row(row: pd.Series) -> None:
    print(row)
    print("\n")
    print("\tEl tipo de dato es : {}".format(type(row)))
    print("\tLos índices son: {}".format(row.index))
    print("\tEl valor con el índice 0 es: {}".format(row[0]))

In [ ]:
_none = nba_df[['Name', 'Team']].head(1).apply(test_by_row, axis = 1)

In [ ]:
row_test = nba_df[['Name', 'Team']].head(1).transpose().squeeze()
row_test

Un momento, ¿Qué ha sucedido?
* `transpose()` traspone el contenido del `DataFrame`, i.e. renglones -> columna
* `squeeze()`, dado que el input es un objeto de la clase `DataFrame` con sólo una columna, transforma su input a un objeto de la clase `Series`

### Eliminar Columnas

El método [`.drop()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.drop.html) se usa para eliminar tanto renglones/filas como columnas. En esta sección veremos cómo usarlo en el caso de las columnas. 



In [ ]:
nba_df.head()

In [ ]:
nba_df.drop(['nombre','apellido'], axis = 1, inplace = True)

In [ ]:
nba_df.head()

El parámetro `axis` nos permite elegir entre aplicar la acción anivel renglones `axis = 0` o columnas `axis = 1`.   


In [ ]:
nba_df.drop([0,2,4], axis = 0).head()

## 2. Gestión Datos Nullos

En esta sección veremos cómo gestionar la ausencia de datos dentro de un DataFrame.
Es muy probable que durante un proceso de análisis de datos lleguemos a encontrar valores faltantes, por ejemplo:

En el DataFrame `nba_df` podemos ver que:

1. La entrada con índice 2 no tiene un valor para la columna `Salary`.
2. La entrada con índice 4 no tiene un valor para la columna `College`.

In [ ]:
nba_df = pd.read_csv("./Data/pandas/nba.csv")
nba_df.head()

La ausencia de un valor dentro de Pandas se representa con el símbolo `NaN` (Not a Number).

De manera general podemos elegir entre tres posibilidades:

1. Eliminarlos
2. Sustituirlos
3. Integrarlos en el proceso como un valor válido.

#### Eliminar - Registros o Columnas 

Para eliminar los registros podemos usar el método [`.dropna()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.dropna.html)

Los parámetros más importantes son:

* `axis`: nos ayuda a elegir entre columnas o renglones
* `how`: {`"any"`, `"all"`}: Determina si un renglón/columna será eliminado si tenemos al menos un NaN (`"any"`) o todos los valores como NaN (`"all"`)

In [ ]:
nba_df.head().dropna(axis = 0, how = "any")

En este caso los registros con índice 2 y 4 fueron eliminados.


In [ ]:
nba_df['ColumnaExtra_1'] = None
nba_df['ColumnaExtra_2'] = np.nan
nba_df['ColumnaExtra_3'] = pd.NA
nba_df.head()

In [ ]:
nba_df.dropna(axis = 1, how = "all").head()

In [ ]:
nba_df.dropna(axis = 1, how = "all", inplace = True)

#### Sustituir

En lugar de eliminar la información podríamos asignar un valor a las entradas faltantes. 

¿Qué valor asignar? La respuesta a esta pregunta dependerá mucho del tipo de datos y el objetivo del análisis, no obstante de manera general podemos aplicar las siguientes reglas:

1. Columna Numérica: Media.
2. Columna String: Moda, valor independiente.

Usaremos el método [`.fillna()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.fillna.html). De entre todos los poarámetros el más importante es `value`: El valor usado para sustituir el valor faltante/nulo. 

In [ ]:
nba_df.head()

In [ ]:
nba_df[['Salary']].fillna(round(nba_df['Salary'].mean(),0)).head()

Supongamos que queremos ejecutar esta acción sobre dos columnas en un sólo comando.

In [ ]:
nba_df = nba_df.apply(
    lambda x: x.fillna(round(x.mean(),0)) if x.name in ["Salary", "Weight"] else x
)

In [ ]:
nba_df['Salary'].isna().any()

En el caso de una columna del tipo `str`, una alternativa podría ser la moda.

In [ ]:
from scipy import stats
nba_df.apply(
    lambda x: x.fillna(stats.mode(x).mode[0]) if x.dtype == "O" else x
).head()

Pero esta elección dependerá mucho del contexto de los datos.

Un dato faltante dentro de la columna `College`, en el contexto de los datos, podría significar que el jugador en cuestión no asistió a la universidad.


Por tanto en este específico escenario, quizás nos interesará más estudiar el efecto de este nivel dentro de la variable `College`.

In [ ]:
nba_df = nba_df.apply(lambda x: x.fillna("No College") if x.name == "College" else x)

In [ ]:
nba_df

## 3. Filtros


### Condiciones Lógicas

Supongamos que del `DataFrame` `nba_df` sólo nos interesan aquellos jugadores con una edad $<= 25$.


In [ ]:
nba_df['Age'] <= 25

Pero habíamos dicho que nos interesa mantener el resto de columnas.

In [ ]:
nba_df[nba_df['Age'] <= 25]

¿Qué pasaría si nos interesa que alguna otra condición también se cumpliera de manera simultánea? Por ejemplo, que `Age<=25` y que `Team=="Boston Celtics`

In [ ]:
nba_df[
    (nba_df['Age']<=25) 
    & (nba_df['Team']=="Boston Celtics")
]

¿Qué pasaría si nos interesaría que al menos alguna de las condiciones se cumpliera? Por ejemplo, que la primera letra de su nombre sea "A" o "B" 

In [ ]:
nba_df[
    (nba_df['Name'].str[0].str.lower()=="a") 
    | (nba_df['Name'].str[0].str.lower()=="b")
].head()

### isin()

El método [`.isin()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.isin.html) es una mejor alternativa.

In [ ]:
nba_df[
    nba_df['Name'].str[0].str.lower().isin(['a','b'])
].head()

### isnull()

Vamos cómo filtrar datos nulos.

In [ ]:
nba_df[nba_df['Age'].isnull()]

Todos los registros con al menos un valor nulo en cualquiera de sus columnas.

In [ ]:
nba_df = pd.read_csv("./Data/pandas/nba.csv")

In [ ]:
nba_df[nba_df.notnull().prod(axis = 1)<1]

¿Cómo contamos los valores nulos por columna?

In [ ]:
pd.DataFrame(nba_df.isnull().sum(axis = 0).sort_values(ascending = False)).reset_index()

### between()

Filtrar aquellos registros con un valor entre dos números[`.between()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.between.html).

In [ ]:
nba_df[nba_df['Age'].between(25,27)]

### query()

El método [`.query()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.query.html) nos permite filtrar usando una condición codificada en formato `str`.

Veamos cómo usarlo:

In [ ]:
nba_df.query("Team == 'Boston Celtics'").head()

Pero el verdadero valor del método `.query()` viene por la opción de llegar a filtrar usando variables.

In [ ]:
col = "Team"
value = 'Boston Celtics'

nba_df.query(f"{col} == '{value}'").head()